In [3]:
import numpy as np
import torch
from tqdm import tqdm
import os

In [13]:
'''
choose between train and val
'''
# filename = 'val'
filename = 'train'

In [14]:
block_size = 256
p = 7
N = 1000000
random_list = np.random.randint(low = 1,high =p+1, size = (N,block_size//2), dtype = np.int32)
input_length_list = np.random.randint(low = 1,high =block_size//2, size = (N,), dtype = np.int8)
token_id_list = []
for i in range(N):
    current_input_tokens = random_list[i][:input_length_list[i]]
    token_id_list.append(list(current_input_tokens) + [-2] + [(current_input_tokens.sum()-1) % p + 1])

In [6]:
token_id_list[1]

[5,
 7,
 6,
 5,
 1,
 5,
 1,
 1,
 4,
 2,
 6,
 2,
 4,
 4,
 1,
 3,
 5,
 4,
 5,
 3,
 4,
 2,
 3,
 5,
 2,
 3,
 5,
 7,
 6,
 6,
 4,
 7,
 1,
 6,
 2,
 5,
 5,
 4,
 6,
 1,
 6,
 1,
 1,
 4,
 6,
 7,
 6,
 6,
 6,
 2,
 5,
 6,
 6,
 5,
 1,
 2,
 5,
 7,
 6,
 1,
 7,
 7,
 5,
 3,
 7,
 7,
 1,
 3,
 4,
 4,
 3,
 5,
 7,
 2,
 4,
 2,
 6,
 4,
 2,
 7,
 6,
 1,
 6,
 6,
 5,
 3,
 7,
 2,
 6,
 6,
 3,
 6,
 5,
 6,
 1,
 5,
 1,
 2,
 4,
 1,
 5,
 4,
 5,
 -2,
 6]

In [15]:
dtype = np.int8
arr = np.memmap(filename + '.bin', dtype=dtype, mode='w+', shape=(block_size * len(token_id_list)))
arr_label = np.memmap(filename + '_label.bin', dtype=dtype, mode='w+', shape=(block_size * len(token_id_list)))
print(f"writing {filename}...")
for jj, example in tqdm(enumerate(token_id_list)):
    arr[jj * block_size : jj * block_size + len(example)-1] = example[:-1]
    arr[jj * block_size + len(example)-1 : (jj + 1) * block_size] = 0
    arr_label[jj * block_size : (1+jj) * block_size ] = -1
    arr_label[jj * block_size + input_length_list[jj]] = example[-1]
    # pad to bock_size
arr.flush()
arr_label.flush()

writing train...


1000000it [00:06, 158220.39it/s]


In [9]:
arr_label

memmap([-1, -1, -1, ...,  0,  0,  0], dtype=int8)

In [5]:
data_dir = os.path.join('data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
train_label = np.memmap(os.path.join(data_dir, 'train_label.bin'), dtype=np.int16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
val_label = np.memmap(os.path.join(data_dir, 'val_label.bin'), dtype=np.int16, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    label = train_label if split == 'train' else val_label
    ix = torch.randint(len(data) // block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i * block_size:(i+1)*block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((label[i * block_size:(i+1)*block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y